In [1]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import pandas as pd
import os
import sys
from tqdm import tqdm

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../scripts")))

from models import Address

df = pd.read_csv("../Data/address-to-geo.csv")
df

,address,lon,lat
0,서울특별시 강동구 명일동 41,127.151841,37.549489
1,서울특별시 양천구 신정동 333,126.873597,37.517713
2,서울특별시 구로구 개봉동 261-10,126.856696,37.490298
3,서울특별시 강동구 길동 320-2,127.142900,37.544728
4,서울특별시 동작구 상도동 264-124,126.936804,37.498164
...,...,...,...
43871,서울특별시 은평구 응암동 97-37,126.925406,37.599768
43872,서울특별시 강서구 화곡동 408-60,126.845355,37.536824
43873,서울특별시 마포구 망원동 432-60,126.901823,37.558830
43874,서울특별시 금천구 독산동 1006-190,126.892263,37.465553


In [ ]:
engine = create_engine(os.getenv("DATABASE_URL"), echo=False)

Session = sessionmaker(bind=engine)
session = Session()

for data in df.itertuples():
    district, legal_dong, lot_number = data.address.split()[1:]
    latitude, longitude = data.lat, data.lon
    lot_number = lot_number.split("-")
    main_lot_number, sub_lot_number = lot_number[0], (
        lot_number[1] if len(lot_number) == 2 else None
    )
    new_address = Address(
        district=district,
        legal_dong=legal_dong,
        main_lot_number=main_lot_number,
        sub_lot_number=sub_lot_number,
        latitude=latitude,
        longitude=longitude,
    )
    session.add(new_address)
session.commit()
session.close()

In [ ]:
from models import BusStation

df = pd.read_csv("../Data/refined-bus.csv")
df

In [ ]:
for data in df.itertuples():
    new_bus_station = BusStation(
        id=data.sttn_no,
        name=data.sttn_name,
        latitude=data.latitude,
        longitude=data.longitude,
    )
    session.add(new_bus_station)
session.commit()
session.close()

In [ ]:
from models import Hospital

df = pd.read_csv("../Data/refined-emergency.csv")
df

In [ ]:
for data in df.itertuples():
    new_hospital = Hospital(
        address=data.address,
        note=data.note,
        map=data.map,
        name=data.name,
        phone=data.phone,
        emergency_phone=data.emergency_phone,
        latitude=data.latitude,
        longitude=data.longitude,
    )
    session.add(new_hospital)
session.commit()
session.close()

In [2]:
from models import Subway

df = pd.read_csv("../Data/refined-subway.csv")
df

,sttn_line,sttn_name,longitude,latitude
0,1호선,서울역,126.972533,37.553150
1,1호선,시청역,126.975407,37.563590
2,1호선,종각역,126.983116,37.570203
3,1호선,종로3가역,126.992095,37.570429
4,1호선,종로5가역,127.001900,37.570971
...,...,...,...,...
271,8호선,남한산성입구역,127.159845,37.451568
272,8호선,단대오거리역,127.156735,37.445057
273,8호선,신흥역,127.147590,37.440952
274,8호선,수진역,127.140936,37.437575


In [ ]:
for data in df.itertuples():
    new_subway = Subway(
        line=data.sttn_line,
        name=data.sttn_name,
        latitude=data.latitude,
        longitude=data.longitude,
    )
    session.add(new_subway)
session.commit()
session.close()

In [ ]:
from models import Building, RealestateDeal, Address

df = (
    pd.read_csv("../Data/refined-real-estate.csv")
    .drop(columns=["Unnamed: 0"])
    .rename(
        columns={
            "건물면적(㎡)": "건물면적",
            "물건금액(만원)": "물건금액",
            "신고한 개업공인중개사 시군구명": "신고한중개사시군구명",
        }
    )
)
df

In [ ]:
for data in tqdm(df.itertuples(), desc="Inserting Data..."):
    sub_lot_number = None if not data.부번 else data.부번
    address_id = (
        session.query(Address)
        .filter_by(
            district=data.자치구명,
            legal_dong=data.법정동명,
            main_lot_number=data.본번,
            sub_lot_number=sub_lot_number,
        )
        .first()
        .id
    )
    existing_building = (
        session.query(Building)
        .filter_by(
            address_id=address_id,
            name=data.건물명,
        )
        .first()
    )
    if existing_building is None:
        new_building = Building(
            address_id=address_id,
            name=data.건물명,
            construction_year=data.건축년도,
            purpose=data.건물용도,
            area_sqm=data.건물면적,
            floor=data.층,
        )
        session.add(new_building)
        session.commit()
        existing_building = (
            session.query(Building)
            .filter_by(
                address_id=address_id,
                name=data.건물명,
            )
            .first()
        )

    new_realestate_del = RealestateDeal(
        building_id=existing_building.id,
        reception_year=data.접수연도,
        transaction_price_million=data.물건금액,
        report_type=data.신고구분,
        reported_real_estate_agent_district=data.신고한중개사시군구명,
        contract_year=data.계약연도,
        contract_month=data.계약월,
        contract_day=data.계약일,
    )
    session.add(new_realestate_del)
session.commit()
session.close()